In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
pip install keras-tuner --upgrade

     |████████████████████████████████| 98 kB 3.7 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense,Embedding,RepeatVector,TimeDistributed,GRU
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from keras.layers import Dropout
from tensorflow import keras
import keras_tuner as kt
 
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [ ]:
# How many sentences will be used
# Limit the sentences to 10.000 on Kaggle to avoid exceding the
# available RAM space
# Build a generator to avoid this issue

total_sentences = 85221

# Load the dataset
dataset = pd.read_excel("/content/drive/MyDrive/data.xlsx", nrows = total_sentences)

# What proportion of the sentences will be used for the test set
test_proportion = 0.3
train_test_threshold = int( (1-test_proportion) * total_sentences)

printmd(f'## {total_sentences} "parallel sentences" will be loaded (original sentence + its translation)')
printmd(f'## {train_test_threshold} "parallel sentences" will be used to train the model')
printmd(f'## {total_sentences-train_test_threshold} "parallel sentences" will be used to test the model')

## 85221 "parallel sentences" will be loaded (original sentence + its translation)

## 59654 "parallel sentences" will be used to train the model

## 25567 "parallel sentences" will be used to test the model

In [ ]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, random_state=0)
dataset.iloc[1000:1010]

,English,PSL
42870,The chief was deciding to be a good man.,Was The chief be a good man decides to now.
3261,I have an orange and an apple.,I an orange an apple have.
73922,I had hoped to meet her there.,was I there her meet hope full.
84184,I know that Shazim is agnostic.,I know that Shazim agnostic.
79760,He is a director and should be treated as such.,He director such treat.
61886,I can not believe Anees said yes.,I believe not was Anees say.
65728,Waasif did not follow the rules.,was Waasif rules follow not.
36340,It sounds very strange to me.,It me very strange sounds.
4064,Balam took off his belt.,was Balam his belt take off.
24879,We have just got to keep going.,We just go keep get full.


In [ ]:
def clean(string):
    # Clean the string
    string = string.replace("\u202f"," ") # Replace no-break space with space
    string = string.lower()
    
    # Delete the punctuation and the numbers
    for p in punctuation + "«»" + "0123456789":
        string = string.replace(p," ")
        
    string = re.sub('\s+',' ', string)
    string = string.strip()
           
    return string

# Clean the sentences
dataset["English"] = dataset["English"].apply(lambda x: clean(x))
dataset["PSL"] = dataset["PSL"].apply(lambda x: clean(x))

# Select one part of the dataset
dataset = dataset.values
dataset = dataset[:total_sentences]

# split into train/test
train, test = dataset[:train_test_threshold], dataset[train_test_threshold:]

# Define the name of the source and of the target
# This will be used in the outputs of this notebook
source_str, target_str = "English", "PSL"

# The index in the numpy array of the source and of the target
idx_src, idx_tar = 0,1

In [ ]:
def create_tokenizer(lines):
    # fit a tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
def max_len(lines):
    # max sentence length
    return max(len(line.split()) for line in lines)

def encode_sequences(tokenizer, length, lines):
    # encode and pad sequences
    X = tokenizer.texts_to_sequences(lines) # integer encode sequences
    X = pad_sequences(X, maxlen=length, padding='post') # pad sequences with 0 values
    return X
 
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y
 
# Prepare target tokenizer
tar_tokenizer = create_tokenizer(dataset[:, idx_tar])
tar_vocab_size = len(tar_tokenizer.word_index) + 1
tar_length = max_len(dataset[:, idx_tar])
printmd(f'\nTarget ({target_str}) Vocabulary Size: {tar_vocab_size}')
printmd(f'Target ({target_str}) Max Length: {tar_length}')

# Prepare source tokenizer
src_tokenizer = create_tokenizer(dataset[:, idx_src])
src_vocab_size = len(src_tokenizer.word_index) + 1
src_length = max_len(dataset[:, idx_src])
printmd(f'\nSource ({source_str}) Vocabulary Size: {src_vocab_size}')
printmd(f'Source ({source_str}) Max Length: {src_length}\n')


Target (PSL) Vocabulary Size: 8089

Target (PSL) Max Length: 14


Source (English) Vocabulary Size: 9256

Source (English) Max Length: 12


## Glove Embedding

In [ ]:
# load embeding for text file
embeddings_index = dict()
f = open('/content/drive/MyDrive/Saved Models/glove.6B.300d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
dims=100
flag=1
word_index = {w: i for i, w in enumerate(embeddings_index, 1)}
embedding_matrix = np.zeros((len(word_index)+1, dims))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector[:dims]

In [ ]:
# Prepare training data
trainX = encode_sequences(src_tokenizer, src_length, train[:8000, idx_src])
trainY = encode_sequences(tar_tokenizer, tar_length, train[:8000, idx_tar])
trainY = encode_output(trainY, tar_vocab_size)

In [ ]:
# Prepare test data
testX = encode_sequences(src_tokenizer, src_length, test[:500, idx_src])
testY = encode_sequences(tar_tokenizer, tar_length, test[:500, idx_tar])
testY = encode_output(testY, tar_vocab_size)

In [ ]:
# #providing the range for hidden layers  
#     for i in range(hp.Int('num_of_layers',1,3)):         
#         #providing range for number of neurons in hidden layers
#         model.add(Dense(units=hp.Int('num_of_neurons'+ str(i),min_value=64,max_value=312,step=64),
#                                     activation='relu'))

In [ ]:
def build_model(hp):


  model = keras.Sequential()
  model.add(Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix],input_length=src_length,trainable=True))
  # 256 is best neuron
  neurons=hp.Int('neurons',min_value=64, max_value=312, step=64)
  model.add(GRU(neurons))
  model.add(RepeatVector(tar_length))
  model.add(GRU(neurons, return_sequences=True))
  
 #providing the range for hidden layers  
  for i in range(hp.Int('num_of_layers',2,4)):         
      #providing range for number of neurons in hidden layers
      model.add(Dense(units=hp.Int('num_of_neurons'+ str(i),min_value=64,max_value=312,step=64),activation='relu'))


  hp_drop=hp.Choice('dropout',values=[0.1,0.2,0.3])
  model.add(Dropout(hp_drop))
  model.add(TimeDistributed(Dense(tar_vocab_size,activation='sigmoid')))
  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])


  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss="categorical_crossentropy",
      metrics=["accuracy"]
  )

  return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective('val_accuracy',direction='max'),
    max_trials=4)
# stop_early=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
stop_early=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10,restore_best_weights=True)


tuner.search(trainX,trainY, epochs=100,validation_data=(testX,testY))
best_model = tuner.get_best_models()[0]

Trial 3 Complete [00h 27m 27s]
val_accuracy: 0.6890000104904175

Best val_accuracy So Far: 0.7261428833007812
Total elapsed time: 01h 21m 25s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
neurons           |256               |192               
num_of_layers     |3                 |2                 
num_of_neurons0   |128               |64                
num_of_neurons1   |128               |64                
dropout           |0.1               |0.1               
learning_rate     |0.0001            |0.0001            
num_of_neurons2   |192               |None              
num_of_neurons3   |64                |None              

Epoch 1/100
250/250 [==============================] - 21s 73ms/step - loss: 5.6542 - accuracy: 0.6228 - val_loss: 3.5781 - val_accuracy: 0.6273
Epoch 2/100
250/250 [==============================] - 16s 64ms/step - loss: 3.1315 - accuracy: 0.6281 - val_loss: 2.8348 - val_accuracy: 0.6273
Epoch 3/100
250/250 [======

In [ ]:
# Tuner Models Summary

tuner.results_summary()

In [ ]:
# Best Parameter
# learning rate: 0.001
# Dropout rate: 0.3
# Neurons: 256